# Compute seasonal error metrics
This notebook computes the RMSE and MAE of a variable for the coarse data (ERA5), reference data, and the downscaled data based on the given station data of a city.

For rainfall, the binary (rain/no-rain days) accuracy is also computed.

In [1]:
import os, sys
from pathlib import Path
import time

import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
import xarray as xr
import xskillscore as xs

from functools import partial
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import glob

import sys
import ast
sys.path.append("../../") 
from src.assess_model import *

In [2]:
RAW_INPUT_PATH = Path("../../data/01-raw/")
INPUT_PATH = Path("../../data/02-processed/")
RESULTS_PATH = Path("../../data/03-results/")
YEARS = np.arange(2003,2023,1)
VARIABLE = 'tmin'
metrics_columns = ['rmse','mae','binary-accuracy']


if VARIABLE == 'precip': 
    evaluation_years = [2008,2009,2010,2016,2017,2018]
    VARIABLE_LR = 'tp'
elif VARIABLE in ['tmin', 'tmax']: 
    evaluation_years = [2008,2009,2010,2016]
    VARIABLE_LR = 't2m_min' if VARIABLE=='tmin' else 't2m_max'
    metrics_columns = metrics_columns[:-1]

# Read data

## Read bounds

In [3]:
bounds_df = pd.read_csv(RAW_INPUT_PATH / 'domains' / 'downscaling_domains_fixed.csv')
bounds_df['full_bounds'] = bounds_df['full_bounds'].apply(ast.literal_eval)
bounds_df['focused_bounds'] = bounds_df['focused_bounds'].apply(ast.literal_eval)
bounds_df.head()

,city,full_bounds,focused_bounds
0,Dagupan,"[120.00931049408791, 120.4821769636825, 15.907...","[120.210342, 120.450668, 15.928978, 16.138177]"
1,Palayan,"[120.8625098711998, 121.33347866342953, 15.294...","[121.042557, 121.131134, 15.495371, 15.584679]"
2,MetroManila,"[120.8470354518582, 121.22452516891933, 14.273...","[120.8670354518582, 121.20452516891933, 14.293..."
3,Legazpi,"[123.63316781038878, 123.90749473184147, 12.97...","[123.68657, 123.767166, 13.113921, 13.205771]"
4,Iloilo,"[122.41374050886832, 122.69006942567574, 10.61...","[122.477646, 122.602422, 10.676429, 10.80698]"


In [4]:
OUTPUT_CITY_NAMES = bounds_df['city'].unique()

## Read station locations

In [5]:
station_locations_df = pd.read_csv(RAW_INPUT_PATH/'station_data'/'PAGASA_station_locations.csv')
station_locations_df = station_locations_df.dropna()
STATION_NAMES = station_locations_df['station_name'].values
STATION_CITY_NAMES = station_locations_df['city_name'].unique()
STATION_CITY_NAMES

array(['Palayan', 'Dagupan', 'Davao', 'CagayanDeOro', 'Legazpi',
       'Mandaue', 'Muntinlupa', 'Navotas', 'Mandaluyong', 'Tacloban',
       'Zamboanga', 'Iloilo'], dtype=object)

## Read downscaled model output

In [6]:
output_ds_list = {}
for city_name in OUTPUT_CITY_NAMES:
    out_ds = xr.load_dataset(RESULTS_PATH / VARIABLE / f"downscaled_{VARIABLE}_{city_name.lower()}_corrected.nc")
    out_ds = out_ds[VARIABLE]
    output_ds_list.update({city_name: out_ds})

## Read reference data

In [7]:
ref_ds_list = {}
for city_name in OUTPUT_CITY_NAMES:
    ref_ds = xr.load_dataset(INPUT_PATH / "model_input" / f"ref_hr_{city_name.lower()}.nc")
    ref_ds = ref_ds[VARIABLE]
    ref_ds_list.update({city_name: ref_ds})

## Read coarse data

In [8]:
lr_ds_list = {}
for city_name in OUTPUT_CITY_NAMES:
    lr_ds = xr.load_dataset(INPUT_PATH / "lr_res" / f"era5_{city_name.lower()}.nc")
    lr_ds = lr_ds[VARIABLE_LR]
    lr_ds_list.update({city_name: lr_ds})

## Read station data

In [9]:
station_ds_list = {}
for city_name in STATION_CITY_NAMES:
    stn_ds = xr.load_dataset(INPUT_PATH / "station_data" / f"pagasa_station_{city_name.lower()}.nc")
    stn_ds = stn_ds[VARIABLE]
    station_ds_list.update({city_name: stn_ds})

In [10]:
station_df = pd.read_csv(INPUT_PATH/'station_data'/ f"station_data_selected_cities.csv")
station_df.head()

,city_name,station_name,date,tmin,tmax,precip
0,Palayan,Cabanatuan,2007-01-01,24.0,32.2,0.0
1,CagayanDeOro,Lumbia,2007-01-01,22.5,28.0,4.8
2,Mandaue,Mactan,2007-01-01,23.7,30.2,6.6
3,Muntinlupa,NAIA,2007-01-01,25.3,31.4,0.0
4,Davao,Davao City,2007-01-01,23.7,26.0,6.0


## Generate metrics

In [11]:
def generate_error_metrics(data1, data2, variable, resample_freq):
    da1_aligned, da2_aligned = xr.align(data1, data2, join='inner')
    # error metrics
    return compute_error_metrics(da1_aligned,da2_aligned,variable,resample_freq)

def generate_season_error_metrics(data1, data2, variable, resample_freq, include_metrics):
    da1_aligned, da2_aligned = xr.align(data1, data2, join='inner')
    # error metrics
    return compute_season_error_metrics(da1_aligned,da2_aligned,variable,resample_freq, include_metrics)

In [12]:
output_metrics_daily_list = []
output_metrics_weekly_list = []
output_metrics_season_list = []

baseline_metrics_daily_list = []
baseline_metrics_weekly_list = []
baseline_metrics_season_list = []

ref_metrics_daily_list = []
ref_metrics_weekly_list = []
ref_metrics_season_list = []

for city_name in STATION_CITY_NAMES:
    print(f'Computing metrics for {city_name}...', end='')
    station_location = station_locations_df[station_locations_df['city_name']==city_name][['lon','lat']].values[0]
    ## Modifications for Zamboanga
    if city_name == 'Zamboanga':
        station_location = np.array([station_location[0],station_location[1]+0.02])
    ## Modifications for Metro Manila
    output_city_name = 'MetroManila' if city_name in ['Muntinlupa', 'Navotas', 'Mandaluyong'] else city_name
    evaluation_years = [2016,2017,2018] if city_name == 'Muntinlupa' else evaluation_years
    # Select only evaluation years
    output_ds=output_ds_list[output_city_name]
    ref_ds=ref_ds_list[output_city_name]
    lr_ds=lr_ds_list[output_city_name]

    output_ds=output_ds.sel(time=output_ds.time.dt.year.isin(evaluation_years))
    ref_ds=ref_ds.sel(time=ref_ds.time.dt.year.isin(evaluation_years))
    lr_ds=lr_ds.sel(time=lr_ds.time.dt.year.isin(evaluation_years))
    # Select only evaluation years for station data
    station_ds = station_ds_list[city_name]
    station_data = station_ds.sel(time=station_ds.time.dt.year.isin(evaluation_years)).mean(dim=['lat', 'lon'], skipna=True)
    # Select station location for other data
    target_lon, target_lat = station_location
    ref_data = ref_ds.sel(lon=target_lon, lat=target_lat, method='nearest')
    lr_data = lr_ds.sel(lon=target_lon, lat=target_lat, method='nearest')
    output_data = output_ds.sel(lon=target_lon, lat=target_lat, method='nearest')
    # lr_lon_bounds, lr_lat_bounds = [lr_data['lon'].values-0.125,lr_data['lon'].values+0.125],[lr_data['lat'].values-0.125,lr_data['lat'].values+0.125]
    # output_data = output_ds.sel(lat=slice(*lr_lat_bounds), lon=slice(*lr_lon_bounds)).mean(dim=['lat', 'lon'], skipna=True)
    #print(f"Output shape: {output_data.shape} Station shape: {station_data.shape}")
    # Compute metrics
    output_metrics_daily = generate_error_metrics(output_data,station_data,VARIABLE,'D')
    output_metrics_weekly = generate_error_metrics(output_data,station_data,VARIABLE,'W')
    output_metrics_season = generate_season_error_metrics(output_data,station_data,VARIABLE,'W', metrics_columns)  
    output_metrics_daily_list.append(output_metrics_daily)
    output_metrics_weekly_list.append(output_metrics_weekly)
    output_metrics_season_list.append(output_metrics_season)

    baseline_metrics_daily = generate_error_metrics(lr_data,station_data,VARIABLE,'D')
    baseline_metrics_weekly = generate_error_metrics(lr_data,station_data,VARIABLE,'W')
    baseline_metrics_season = generate_season_error_metrics(lr_data,station_data,VARIABLE,'W', metrics_columns)   
    baseline_metrics_daily_list.append(baseline_metrics_daily)
    baseline_metrics_weekly_list.append(baseline_metrics_weekly)
    baseline_metrics_season_list.append(baseline_metrics_season)

    ref_metrics_daily = generate_error_metrics(ref_data,station_data,VARIABLE,'D')
    ref_metrics_weekly = generate_error_metrics(ref_data,station_data,VARIABLE,'W')
    ref_metrics_season = generate_season_error_metrics(ref_data,station_data,VARIABLE,'W', metrics_columns)   
    ref_metrics_daily_list.append(ref_metrics_daily)
    ref_metrics_weekly_list.append(ref_metrics_weekly)
    ref_metrics_season_list.append(ref_metrics_season)
    print('DONE!')

Computing metrics for Palayan...DONE!
Computing metrics for Dagupan...

/home/jace/miniconda3/envs/climate-analytics/lib/python3.10/site-packages/xskillscore/core/np_deterministic.py:263: RuntimeWarning: invalid value encountered in scalar divide
  r2 = 1 - (num / den)
/home/jace/miniconda3/envs/climate-analytics/lib/python3.10/site-packages/xskillscore/core/np_deterministic.py:263: RuntimeWarning: invalid value encountered in scalar divide
  r2 = 1 - (num / den)
/home/jace/miniconda3/envs/climate-analytics/lib/python3.10/site-packages/xskillscore/core/np_deterministic.py:263: RuntimeWarning: invalid value encountered in scalar divide
  r2 = 1 - (num / den)
/home/jace/miniconda3/envs/climate-analytics/lib/python3.10/site-packages/xskillscore/core/np_deterministic.py:263: RuntimeWarning: invalid value encountered in scalar divide
  r2 = 1 - (num / den)
/home/jace/miniconda3/envs/climate-analytics/lib/python3.10/site-packages/xskillscore/core/np_deterministic.py:263: RuntimeWarning: invalid value encountered in scalar divide
  r2 = 1 - (num / den)
/home/jace

DONE!
Computing metrics for Davao...DONE!
Computing metrics for CagayanDeOro...DONE!
Computing metrics for Legazpi...DONE!
Computing metrics for Mandaue...DONE!
Computing metrics for Muntinlupa...DONE!
Computing metrics for Navotas...DONE!
Computing metrics for Mandaluyong...DONE!
Computing metrics for Tacloban...DONE!
Computing metrics for Zamboanga...

/home/jace/miniconda3/envs/climate-analytics/lib/python3.10/site-packages/xskillscore/core/np_deterministic.py:263: RuntimeWarning: invalid value encountered in scalar divide
  r2 = 1 - (num / den)
/home/jace/miniconda3/envs/climate-analytics/lib/python3.10/site-packages/xskillscore/core/np_deterministic.py:263: RuntimeWarning: invalid value encountered in scalar divide
  r2 = 1 - (num / den)
/home/jace/miniconda3/envs/climate-analytics/lib/python3.10/site-packages/xskillscore/core/np_deterministic.py:263: RuntimeWarning: invalid value encountered in scalar divide
  r2 = 1 - (num / den)
/home/jace/miniconda3/envs/climate-analytics/lib/python3.10/site-packages/xskillscore/core/np_deterministic.py:263: RuntimeWarning: invalid value encountered in scalar divide
  r2 = 1 - (num / den)
/home/jace/miniconda3/envs/climate-analytics/lib/python3.10/site-packages/xskillscore/core/np_deterministic.py:263: RuntimeWarning: invalid value encountered in scalar divide
  r2 = 1 - (num / den)
/home/jace

DONE!
Computing metrics for Iloilo...DONE!


In [13]:
# Format metric tables
print('Processing metric tables...', end='')

output_metrics_daily_df = pd.concat(output_metrics_daily_list)
output_metrics_daily_df['city_name'] = STATION_CITY_NAMES
output_metrics_daily_df = output_metrics_daily_df[['city_name']+metrics_columns]
baseline_metrics_daily_df = pd.concat(baseline_metrics_daily_list)
baseline_metrics_daily_df['city_name'] = STATION_CITY_NAMES
baseline_metrics_daily_df = baseline_metrics_daily_df[['city_name']+metrics_columns]
ref_metrics_daily_df = pd.concat(ref_metrics_daily_list)
ref_metrics_daily_df['city_name'] = STATION_CITY_NAMES
ref_metrics_daily_df = ref_metrics_daily_df[['city_name']+metrics_columns]

output_metrics_weekly_df = pd.concat(output_metrics_weekly_list)
output_metrics_weekly_df['city_name'] = STATION_CITY_NAMES
output_metrics_weekly_df = output_metrics_weekly_df[['city_name']+metrics_columns]
baseline_metrics_weekly_df = pd.concat(baseline_metrics_weekly_list)
baseline_metrics_weekly_df['city_name'] = STATION_CITY_NAMES
baseline_metrics_weekly_df = baseline_metrics_weekly_df[['city_name']+metrics_columns]
ref_metrics_weekly_df = pd.concat(ref_metrics_weekly_list)
ref_metrics_weekly_df['city_name'] = STATION_CITY_NAMES
ref_metrics_weekly_df = ref_metrics_weekly_df[['city_name']+metrics_columns]

output_metrics_season_df = pd.concat(output_metrics_season_list, axis=0)
output_metrics_season_df['city_name'] = STATION_CITY_NAMES
baseline_metrics_season_df = pd.concat(baseline_metrics_season_list)
baseline_metrics_season_df['city_name'] = STATION_CITY_NAMES
ref_metrics_season_df = pd.concat(ref_metrics_season_list)
ref_metrics_season_df['city_name'] = STATION_CITY_NAMES
print('DONE!')


Processing metric tables...DONE!


In [14]:
output_metrics_weekly_df

,city_name,rmse,mae
0,Palayan,0.933424,0.769731
0,Dagupan,1.150215,1.023588
0,Davao,3.732103,3.676391
0,CagayanDeOro,5.644860,5.557185
0,Legazpi,1.414080,1.316227
0,Mandaue,0.420862,0.332757
0,Muntinlupa,1.511333,1.388744
0,Navotas,1.799217,1.693331
0,Mandaluyong,2.753385,2.626154
0,Tacloban,0.784441,0.675411


In [15]:
baseline_metrics_weekly_df

,city_name,rmse,mae
0,Palayan,0.478257,0.368446
0,Dagupan,1.967028,1.818091
0,Davao,1.065706,0.931706
0,CagayanDeOro,2.770219,2.634600
0,Legazpi,0.897662,0.808638
0,Mandaue,0.825551,0.719683
0,Muntinlupa,0.823757,0.712239
0,Navotas,0.888750,0.786879
0,Mandaluyong,1.752172,1.533198
0,Tacloban,1.308982,1.247354


In [16]:
output_metrics_season_df #bias corrected

,rmse_DJF,mae_DJF,rmse_JJA,mae_JJA,rmse_MAM,mae_MAM,rmse_SON,mae_SON,city_name
0,0.722550,0.521698,0.836468,0.741249,1.253155,1.101154,0.824280,0.707054,Palayan
0,0.933881,0.799214,1.270224,1.180325,1.129749,1.004991,1.234420,1.106280,Dagupan
0,3.375422,3.339801,3.967933,3.924593,3.424031,3.381337,4.106384,4.061034,Davao
0,4.999517,4.925617,6.018285,5.949754,5.782533,5.722465,5.714293,5.616602,CagayanDeOro
0,1.207566,1.076190,1.559859,1.491585,1.463999,1.393424,1.396074,1.297235,Legazpi
0,0.408792,0.328736,0.433851,0.334627,0.445406,0.371220,0.392361,0.295411,Mandaue
0,1.122243,0.946169,1.720724,1.639043,1.575293,1.483453,1.577083,1.509009,Muntinlupa
0,1.384359,1.274479,2.076028,2.021004,1.833554,1.699276,1.851538,1.800045,Navotas
0,2.713368,2.650948,2.171738,2.056040,3.387381,3.292091,2.604339,2.504263,Mandaluyong
0,0.633519,0.538837,0.971011,0.866174,0.670064,0.581357,0.824116,0.722282,Tacloban


In [17]:
baseline_metrics_season_df

,rmse_DJF,mae_DJF,rmse_JJA,mae_JJA,rmse_MAM,mae_MAM,rmse_SON,mae_SON,city_name
0,0.593430,0.438659,0.434911,0.371674,0.475025,0.354463,0.386078,0.309264,Palayan
0,2.630828,2.506571,1.502386,1.425812,1.899059,1.799645,1.655303,1.550868,Dagupan
0,0.706113,0.634513,1.250994,1.162041,0.830442,0.690269,1.337958,1.240286,Davao
0,2.570971,2.459973,2.887986,2.753283,2.861902,2.731272,2.743225,2.588349,CagayanDeOro
0,0.751818,0.659518,0.822073,0.772777,1.148215,1.053902,0.808671,0.742984,Legazpi
0,0.983910,0.872826,0.749630,0.675166,0.587480,0.502545,0.928802,0.834904,Mandaue
0,0.928555,0.829463,0.827002,0.694427,0.714034,0.618215,0.805538,0.700841,Muntinlupa
0,0.958997,0.849201,0.992816,0.915143,0.728786,0.624147,0.846384,0.755829,Navotas
0,2.200069,2.137481,0.801924,0.665231,2.085417,1.958533,1.535350,1.340560,Mandaluyong
0,1.476240,1.429251,1.181829,1.131419,1.234482,1.163834,1.315257,1.255584,Tacloban
